In [3]:
import pandas as pd
import os
from striprtf.striprtf import rtf_to_text

file_names = os.listdir('./SleepMeditations/')
# Create Dictionary for File Name and Text
file_name_and_text = {}
for file in file_names:
    if (file == ".ipynb_checkpoints"):
        continue

    with open('../SleepMeditations/' + file, "r",
             encoding='unicode_escape') as infile:
        content = infile.read()
        #text = rtf_to_text(content)
        file_name_and_text[file] = content
file_data = (pd.DataFrame.from_dict(file_name_and_text, orient='index')
             .reset_index().rename(index=str, columns={'index': 'file_name', 0: 'text'}))

file_data

FileNotFoundError: [Errno 2] No such file or directory: '../SleepMeditations/Sleep - 33.txt'

In [19]:
import math
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anushree/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
en_stop_words = set(stopwords.words('english'))
list(en_stop_words)[:10]

def clean_tweets(df=file_data, 
                 text_col='text', 
                ):
    
    df_copy = df.copy()
    
    # drop rows with empty values
    df_copy.dropna(inplace=True)
    
    # format the date
    #df_copy[date_col] = df_copy[date_col].apply(lambda row: datetime.strptime(row, '%m-%d-%Y %H:%M:%S'))
    
    # filter rows older than a given date
    #df_copy = df_copy[df_copy[date_col] >=start_datetime]
    
    # lower the tweets
    df_copy['preprocessed_' + text_col] = df_copy[text_col].str.lower()
    
    # filter out stop words and URLs
    en_stop_words = set(stopwords.words('english'))
    en_stop_words.add('music')
    en_stop_words.add('Jason Stevenson')
    extended_stop_words = en_stop_words | \
                        {
                            '&amp;', 'rt',                           
                            'th','co', 're', 've', 'kim', 'daca'
                        }
    url_re = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'        
    df_copy['preprocessed_' + text_col] = df_copy['preprocessed_' + text_col].apply(lambda row: ' '.join(
        [word for word in row.split() if (not word in en_stop_words) and (not re.match(url_re, word))]))
    
    # tokenize the tweets
    tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
    df_copy['tokenized_' + text_col] = df_copy['preprocessed_' + text_col].apply(lambda row: tokenizer.tokenize(row))
    
    return df_copy

#df_tweets = pd.read_csv('trump_tweets.csv')
df_clean = clean_tweets(file_data)
df_clean.head()

NameError: name 'file_data' is not defined

In [7]:
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(str)
    bag_of_words = vect.transform(str)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
get_most_freq_words([ word for line in df_clean.tokenized_text for word in line],10)

NameError: name 'df_clean' is not defined

In [8]:
import matplotlib.pyplot as plt
# build a dictionary where for each tweet, each word has its own id.
# We have 6882 tweets and 10893 words in the dictionary.
tweets_dictionary = Dictionary(df_clean.tokenized_text)

# build the corpus i.e. vectors with the number of occurence of each word per tweet
tweets_corpus = [tweets_dictionary.doc2bow(
    tweet) for tweet in df_clean.tokenized_text]

# compute coherence
tweets_coherence = []
for nb_topics in range(1, 36):
    lda = LdaModel(tweets_corpus, num_topics=nb_topics,
                   id2word=tweets_dictionary, passes=10)
    cohm = CoherenceModel(model=lda, corpus=tweets_corpus,
                          dictionary=tweets_dictionary, coherence='u_mass')
    coh = cohm.get_coherence()
    tweets_coherence.append(coh)

# visualize coherence
plt.figure(figsize=(10, 5))
plt.plot(range(1, 36), tweets_coherence)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")



NameError: name 'df_clean' is not defined

In [9]:
import matplotlib.gridspec as gridspec

k = 6
tweets_lda = LdaModel(tweets_corpus, num_topics=k,
                      id2word=tweets_dictionary, passes=10)


def plot_top_words(lda=tweets_lda, nb_topics=k, nb_words=10):
    top_words = [[word for word, _ in lda.show_topic(
        topic_id, topn=50)] for topic_id in range(lda.num_topics)]
    top_betas = [[beta for _, beta in lda.show_topic(
        topic_id, topn=50)] for topic_id in range(lda.num_topics)]

    gs = gridspec.GridSpec(round(math.sqrt(k))+1, round(math.sqrt(k))+1)
    gs.update(wspace=0.5, hspace=0.5)
    plt.figure(figsize=(20, 15))
    for i in range(nb_topics):
        ax = plt.subplot(gs[i])
        plt.barh(range(nb_words), top_betas[i][:nb_words],
                 align='center', color='blue', ecolor='black')
        ax.invert_yaxis()
        ax.set_yticks(range(nb_words))
        ax.set_yticklabels(top_words[i][:nb_words])
        plt.title("Topic "+str(i))


plot_top_words()

NameError: name 'tweets_corpus' is not defined